In [1]:
!pip install -e /Users/lars/devel/tpk4170-robotics/

Obtaining file:///Users/lars/devel/tpk4170-robotics
  Found existing installation: tpk4170 1.0
    Uninstalling tpk4170-1.0:
      Successfully uninstalled tpk4170-1.0
  Running setup.py develop for tpk4170


In [1]:
import numpy as np
from tpk4170.models import ColladaMesh, Grid, Axes
from tpk4170.visualization import Viewer
from tpk4170.utils.transformations import quaternion_from_euler

In [2]:
from pythreejs import Object3D
from enum import Enum

In [35]:
class Visual(Object3D):
    def __init__(self, geometry=None, origin=[0.,0.,0.], rpy=[0.0,0.0,0.0]):
        Object3D.__init__(self)
        self.add(geometry)
        self.position = origin
        self.quaternion = quaternion_from_euler(*rpy).tolist()
        
class Link(Object3D):
    def __init__(self, visual, origin=[0.,0.,0.], rpy=[0.0,0.0,0.0]):
        Object3D.__init__(self)
        self.visual = visual
        self.add(self.visual)
        
        self.position = origin
        self.quaternion = quaternion_from_euler(*rpy).tolist()

        
class Joint(Object3D):
    def __init__(self, parent, child, axis, type_, origin=[0.,0.,0.], rpy=[0.0,0.0,0.0]):
        Object3D.__init__(self)
        self.parent = parent
        self.child = child
        self.axis = axis
        self.type_ = type_
        
        self.parent.add(self)
        self.add(child)
        
        self.position = origin
        self.quaternion = quaternion_from_euler(*rpy).tolist()
        
        self._q = 0.0
        
    def angle(self, theta):
        self.child.quaternion = quaternion_from_euler(*(np.array(self.axis) * theta)).tolist()
        

In [58]:
base_link = Link(visual=Visual(geometry=ColladaMesh('./robotiq/2f85/robotiq_85_base_link.dae')))

left_knuckle_link = Link(visual=Visual(geometry=ColladaMesh('./robotiq/2f85/robotiq_85_knuckle_link.dae')))
right_knuckle_link = Link(visual=Visual(geometry=ColladaMesh('./robotiq/2f85/robotiq_85_knuckle_link.dae')))

left_knuckle_joint = Joint(parent=base_link, 
                           child=left_knuckle_link, 
                           axis=[0.0, 0.0, 1.0], 
                           type_='revolute',
                           origin=[0.05490451627, 0.03060114443, 0.0],
                           rpy=[np.pi, 0.0, 0.0])

right_knuckle_joint = Joint(parent=base_link, 
                           child=right_knuckle_link, 
                           axis=[0.0, 0.0, 1.0], 
                           type_='revolute',
                           origin=[0.05490451627, -0.03060114443, 0.0],
                           rpy=[0.0, 0.0, 0.0])


left_finger_link = Link(visual=Visual(geometry=ColladaMesh('./robotiq/2f85/robotiq_85_finger_link.dae')))
right_finger_link = Link(visual=Visual(geometry=ColladaMesh('./robotiq/2f85/robotiq_85_finger_link.dae')))

left_finger_joint = Joint(parent=left_knuckle_link, 
                           child=left_finger_link, 
                           axis=[0.0, 0.0, 1.0], 
                           type_='revolute',
                           origin=[-0.00408552455, -0.03148604435, 0.0],
                           rpy=[0.0, 0.0, 0.0])

right_finger_joint = Joint(parent=right_knuckle_link, 
                           child=right_finger_link, 
                           axis=[0.0, 0.0, 1.0], 
                           type_='revolute',
                           origin=[-0.00408552455, -0.03148604435, 0.0],
                           rpy=[0.0, 0.0, 0.0])

left_inner_knuckle_link = Link(visual=Visual(
    geometry=ColladaMesh('./robotiq/2f85/robotiq_85_inner_knuckle_link.dae')))
right_inner_knuckle_link = Link(visual=Visual(
    geometry=ColladaMesh('./robotiq/2f85/robotiq_85_inner_knuckle_link.dae')))

left_inner_knuckle_joint = Joint(parent=base_link, 
                                 child=left_inner_knuckle_link, 
                                 axis=[0.0, 0.0, 1.0], 
                                 type_='revolute',
                                 origin=[0.06142, 0.0127, 0.0],
                                 rpy=[np.pi, 0.0, 0.0])

right_inner_knuckle_joint = Joint(parent=base_link, 
                                  child=right_inner_knuckle_link, 
                                  axis=[0.0, 0.0, 1.0], 
                                  type_='revolute',
                                  origin=[0.06142, -0.0127, 0.0],
                                  rpy=[0.0, 0.0, 0.0])

left_finger_tip_link = Link(visual=Visual(
    geometry=ColladaMesh('./robotiq/2f85/robotiq_85_finger_tip_link.dae')))
right_finger_tip_link = Link(visual=Visual(
    geometry=ColladaMesh('./robotiq/2f85/robotiq_85_finger_tip_link.dae')))

left_finger_tip_joint = Joint(parent=left_inner_knuckle_link, 
                                 child=left_finger_tip_link, 
                                 axis=[0.0, 0.0, 1.0], 
                                 type_='revolute',
                                 origin=[0.04303959807, -0.03759940821, 0.0],
                                 rpy=[0.0, 0.0, 0.0])

right_finger_tip_joint = Joint(parent=right_inner_knuckle_link, 
                                  child=right_finger_tip_link, 
                                  axis=[0.0, 0.0, 1.0], 
                                  type_='revolute',
                                  origin=[0.04303959807, -0.03759940821, 0.0],
                                  rpy=[0.0, 0.0, 0.0])


# Link(visual=ColladaMesh('./robotiq/2f85/robotiq_85_knuckle_link.dae'))
# right_knuckle_link = Link(visual=ColladaMesh('./robotiq/2f85/robotiq_85_knuckle_link.dae'))
# left_knuckle_joint = Joint(parent=base_link, child=left_knuckle_link)

In [59]:
# <xacro:macro name="inner_knuckle_joint" params="prefix fingerprefix reflect">
#     <joint name="${prefix}${fingerprefix}_inner_knuckle_joint" type="revolute">
#       <origin xyz="0 ${reflect * -0.0127} 0.06142" rpy="${pi / 2 + .725} 0 ${(reflect - 1) * pi / 2}" />
#       <parent link="${prefix}robotiq_arg2f_base_link" />
#       <child link="${prefix}${fingerprefix}_inner_knuckle" />
#       <axis xyz="1 0 0" />
#       <limit lower="0" upper="0.8757" velocity="2.0" effort="1000" />
#       <mimic joint="${prefix}finger_joint" multiplier="-1" offset="0" />
#     </joint>
#   </xacro:macro>

In [60]:
base_link

Preview(child=Link(children=(Visual(children=(ColladaMesh(children=(Mesh(geometry=BufferGeometry(attributes={'…

In [61]:
viewer = Viewer(background='white')
viewer.add(Grid())
viewer.add(base_link)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [66]:
left_knuckle_joint.angle(0.2)
left_finger_joint.angle(-0.2)
left_inner_knuckle_joint.angle(0.2)
left_finger_tip_joint.angle(-0.2)

In [53]:
left_knuckle_joint.angle(0.0)

, left_finger_tip_joint]
for j in ljoints:
    j.angle(0.0)

In [23]:
left_knuckle_link.quaternion = quaternion_from_euler(np.pi,0.0,0.0).tolist()
left_knuckle_link.position = [0.05490451627, 0.03060114443, 0.0]
base_link.add(left_knuckle_link)

In [24]:
# right_knuckle_link.quaternion = quaternion_from_euler(0,0.0,0.0).tolist()
right_knuckle_link.position = [0.05490451627, -0.03060114443, 0.0]
base_link.add(right_knuckle_link)

In [25]:
left_finger_link.position = [-0.00408552455, -0.03148604435, 0.0]
left_knuckle_link.add(left_finger_link)

In [26]:
right_finger_link.position = [-0.00408552455, -0.03148604435, 0.0]
right_knuckle_link.add(right_finger_link)

In [27]:
left_inner_knuckle_link.position = [0.06142, 0.0127, 0.0]
left_inner_knuckle_link.quaternion = quaternion_from_euler(np.pi, 0.0, 0.0).tolist()
base_link.add(left_inner_knuckle_link)

In [28]:
right_inner_knuckle_link.position = [0.06142, -0.0127, 0.0]
base_link.add(right_inner_knuckle_link)

In [19]:
left_finger_tip_link.position = [0.04303959807, -0.03759940821, 0.0]
left_inner_knuckle_link.add(left_finger_tip_link)

In [20]:
right_finger_tip_link.position = [0.04303959807, -0.03759940821, 0.0]
right_inner_knuckle_link.add(right_finger_tip_link)